In [ ]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import momepy
import folium
import matplotlib.pyplot as plt
from contextily import add_basemap
from libpysal import weights

In [ ]:
rues = pd.read_csv('data/rues.csv', sep=';')

In [ ]:
rues.info()

In [ ]:
cyclables = pd.read_csv('data/cyclables.csv', sep=';')

In [ ]:
cyclables.head()

In [ ]:
merged = pd.merge(rues, cyclables, on='gid', how='left', suffixes=('_rue', '_cyclable'))

In [ ]:
merged.info()

In [ ]:
rues_geojson = gpd.read_file('data/rues.geojson')
cyclables_geojson = gpd.read_file('data/cyclables.geojson')

In [ ]:
rues_geojson.info()

In [ ]:
rues_geojson.head()

In [ ]:
cyclables_geojson.info()

In [ ]:
rues_geojson = rues_geojson[['geometry']]
cyclables_geojson = cyclables_geojson[['geometry']]


In [ ]:
rues_geojson.loc[:,'label'] = 'rue'
cyclables_geojson.loc[:,'label'] = 'cyclable'

In [ ]:
rues_geojson.head()

In [ ]:
cyclables_geojson = cyclables_geojson.dropna(subset=['geometry'])

In [ ]:
merged_geojson = gpd.sjoin(cyclables_geojson, rues_geojson, how='left')

In [ ]:
merged_geojson.head()

In [ ]:
merged_geojson.info()

In [ ]:
# print unique value of label_left
merged_geojson['label_left'].unique()

In [ ]:
map = folium.Map(location=[44.833328,-0.56667], zoom_start=13,tiles="OpenStreetMap")

In [ ]:
for index,r in merged_geojson.iterrows():
    if index == 1:
        break
    folium.GeoJson(r['geometry']).add_to(map)
    print("ok")

In [ ]:
map

In [ ]:
merged_geojson.info()

In [ ]:
map = folium.Map(location=[44.833328,-0.56667], zoom_start=13,tiles="OpenStreetMap")

In [ ]:
merged_geojson.head()

In [ ]:
for _,r in merged_geojson.iterrows():
    if r.label_right == 'cyclable':
        folium.GeoJson(r.geometry, style_function=lambda x: {'color': 'red'}).add_to(map)
    else:
        folium.GeoJson(r.geometry, style_function=lambda x: {'color': 'blue'}).add_to(map)

In [ ]:
map

In [ ]:
merged_geojson['geometry'].apply(lambda x: x.geom_type).unique()

In [ ]:
merged_geojson[merged_geojson['geometry'].apply(lambda x: x.geom_type) == 'MultiLineString'].plot()

In [ ]:
test_merged = merged_geojson.explode()

In [ ]:
test_merged.info()

In [ ]:
test_merged['geometry'].apply(lambda x: x.geom_type).unique()

In [ ]:
new_map = folium.Map(location=[44.833328,-0.56667], zoom_start=13,tiles="OpenStreetMap")

In [ ]:
for _,r in test_merged.iterrows():
    if r.label_right == 'cyclable':
        folium.GeoJson(r.geometry, style_function=lambda x: {'color': 'red'}).add_to(new_map)
    else:
        folium.GeoJson(r.geometry).add_to(new_map)

In [ ]:
new_map

In [ ]:
ru = momepy.gdf_to_nx([:100], approach="primal")

In [ ]:
positions = {n: [n[0], n[1]] for n in list(ru.nodes)}

# Plot
f, ax = plt.subplots(1, 2, figsize=(64, 32), sharex=True, sharey=True)
rues_geojson.plot(color="k", ax=ax[0])
for i, facet in enumerate(ax):
    facet.set_title(("Rivers", "Graph")[i])
    facet.axis("off")
nx.draw(ru, positions, ax=ax[1], node_size=5)

In [ ]:
nodes, edges, W = momepy.nx_to_gdf(ru, spatial_weights=True)


# Read in example street network from GeoPackage
streets = gpd.read_file(, layer="streets")

# Construct the primal graph
G_primal = momepy.gdf_to_nx(streets, approach="primal")

# Plot
f, ax = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)
streets.plot(color="k", ax=ax[0])
for i, facet in enumerate(ax):
    facet.set_title(("Streets", "Graph")[i])
    facet.axis("off")
    try:  # For issues with downloading/parsing in CI
        add_basemap(facet)
    except:
        pass
nx.draw(
    G_primal, {n: [n[0], n[1]] for n in list(G_primal.nodes)}, ax=ax[1], node_size=50
)